In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

"""
Load Vehicle recall

"""

# Import libraries

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import aerospike
import json
import csv

print(__doc__)

def load():
    
    global rec
    rec = {}

    csvfile  = open('/home/azureuser/Aerospike/data/customer/vehicle_recall.csv', "rb")
    reader = csv.reader(csvfile)

    config = {
                'hosts': [ ('127.0.0.1', 3000) ]
            }

    # Create a client and connect it to the cluster
    try:
        client = aerospike.client(config).connect()
    except:
        import sys
        print("failed to connect to the cluster with", config['hosts'])
        sys.exit(1)
            
    rownum = 0
    for row in reader:
        # Save First Row with headers
        if rownum == 0:
            header = row
            header = [x.lower() for x in header]
        else:
            colnum = 0
            for col in row:
                # print (rownum,header[colnum],col)
                rec[header[colnum]] = col
                colnum += 1
        rownum += 1
        # print(rownum,rec)
        if rec:
            try:
                client = aerospike.client(config).connect()
                # Write a record
                client.put(('dev', 'recall', str(rownum)), rec)
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)
        
            # Read a record after insert for verification
            (key, metadata, record) = client.get(('dev', 'recall', str(rownum)))
            print (key, record)
        
            # Close the connection to the Aerospike cluster
            client.close()
        rec = {}

def delete():
    
    global rec
    rec = {}

    csvfile  = open('/home/azureuser/Aerospike/data/customer/vehicle_recall.csv', "rb")
    reader = csv.reader(csvfile)

    config = {
                'hosts': [ ('127.0.0.1', 3000) ]
            }

    # Create a client and connect it to the cluster
    try:
        client = aerospike.client(config).connect()
    except:
        import sys
        print("failed to connect to the cluster with", config['hosts'])
        sys.exit(1)
            
    rownum = 0
    for row in reader:
        # Save First Row with headers
        if rownum == 0:
            header = row
            header = [x.lower() for x in header]
        else:
            colnum = 0
            for col in row:
                # print (rownum,header[colnum],col)
                rec[header[colnum]] = col
                colnum += 1
        rownum += 1
        # print(rownum,rec)
        if rec:
            try:
                client = aerospike.client(config).connect()
                # Write a record
                client.remove(('dev', 'recall', str(rownum)))
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)
        
            # Close the connection to the Aerospike cluster
            client.close()
        rec = {}

def drive(mode):
    if mode == 'load':
        load()
    else:
        delete()
        
if __name__ == '__main__':
    drive('load')


Load Vehicle recall


('dev', 'recall', None, bytearray(b'4\x14\xd5\x80\xb2\x1a\xfcD\xfe\x88\xc5\xf83\x12\x0exE\xc4\x18R')) {'model_type': 'ML320 Diesel', 'to': '2010', 'model': 'M-Class', 'affected': 'Driver-Side Airbags', 'from': '2009'}
('dev', 'recall', None, bytearray(b'pB\xbal\xff\x8c78\xeb\xec{hc\xb0\xba\x83\xdai\xa5d')) {'model_type': 'ML350', 'to': '2011', 'model': 'M-Class', 'affected': 'Driver-Side Airbags', 'from': '2009'}
('dev', 'recall', None, bytearray(b'\xbdC\xf3^\x9a\xbd\xbfc\xbd!\xbc\xab\xd5\xcd\xa1y}\xfehZ')) {'model_type': 'ML450 Hybrid', 'to': '2011', 'model': 'M-Class', 'affected': 'Driver-Side Airbags', 'from': '2010'}
('dev', 'recall', None, bytearray(b'\x16*\x9eK@+\x18|\x15\xa25\xdc\x82\xec\x08\xe4\xf4\xf2\xdbX')) {'model_type': 'ML550', 'to': '2017', 'model': 'M-Class', 'affected': 'Driver-Side Airbags', 'from': '2017'}
('dev', 'recall', None, bytearray(b'eO5\t\x19\xfd\x03B\xd0\x107;B\x9b\xa6\x01\x13u/D')) {'model_type': 'AMG ML63', 'to': '2011', 'model': 'M

### Create index for table dev.recall

Syntax - CREATE INDEX numindex ON test.testset (binB) NUMERIC|STRING

###### Index 'model_type'
create index i_model_type on dev.recall (model_type) string